In [1]:
import os
import sys

In [3]:
pwd

'c:\\Users\\arpit\\OneDrive\\Desktop\\Chicken-Diseases-Classification-with-Fecal-Images\\research'

In [4]:
os.chdir("../")

In [5]:
pwd

'c:\\Users\\arpit\\OneDrive\\Desktop\\Chicken-Diseases-Classification-with-Fecal-Images'

In [10]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("allandclive/chicken-disease-1")

print("Path to dataset files:", path)

c:\Users\arpit\anaconda3\envs\tensor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 266M/266M [00:43<00:00, 6.39MB/s] 

Extracting files...


Path to dataset files: C:\Users\arpit\.cache\kagglehub\datasets\allandclive\chicken-disease-1\versions\1


In [13]:
import shutil
import os

# Define source and destination paths
source_path = r"C:\Users\arpit\.cache\kagglehub\datasets\allandclive\chicken-disease-1\versions\1"
destination_path = r"C:\Users\arpit\OneDrive\Desktop\Chicken-Diseases-Classification-with-Fecal-Images\data"

# Ensure the destination directory exists
os.makedirs(destination_path, exist_ok=True)

# Copy all files and folders from source to destination
shutil.move(source_path, destination_path)

print(f"Dataset moved successfully to {destination_path}")

Dataset moved successfully to C:\Users\arpit\OneDrive\Desktop\Chicken-Diseases-Classification-with-Fecal-Images\data


In [ ]:
dataset_link = "https://github.com/ArpitKadam/Chicken-Diseases-Classification-with-Fecal-Images/raw/refs/heads/main/Chicken-fecal-images.zip"

# DATA INGESTION STEP

### Entity

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### Configuration Manager

In [15]:
from src.constants import *
from src.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

### Components

In [16]:
import os
import urllib.request as request
import zipfile
from src.logger.logger import logger
from src.utils.common import get_size
from pathlib import Path


class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info("Attempt to download data")
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        logger.info(f"Unzipping file from {self.config.local_data_file} into {unzip_path}")
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Unzipping completed")
        logger.info(f"File successfully extracted to {unzip_path}")

### Pipeline

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-03-19 16:02:02,375: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-19 16:02:02,379: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-19 16:02:02,381: INFO: common: created directory at: artifacts]
[2025-03-19 16:02:02,384: INFO: common: created directory at: artifacts/data_ingestion]
[2025-03-19 16:02:02,386: INFO: 2904967687: Attempt to download data]
[2025-03-19 16:02:12,500: INFO: 2904967687: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11616915
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "adf745abc03891fe493c3be264ec012691fe3fa21d861f35a27edbe6d86a76b1"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 9EC7:1CC0FF:26623:3F484:67DA9D20
Accept-Ranges: bytes
Date: Wed, 19 Mar 2025 10